# Esercizio 1b

Implementare l’algoritmo di Lesk (!= usare implementazione esistente, e.g., in nltk…).
- Estrarre 50 frasi dal corpus SemCor (corpus annotato con i synset di WN) e disambiguare (almeno) un sostantivo per frase. Calcolare l’accuratezza del sistema implementato sulla base dei sensi annotati in SemCor (SemCor è disponibile all’URL http://web.eecs.umich.edu/~mihalcea/downloads.html)
- Randomizzare la selezione delle 50 frasi e la selezione del termine da disambiguare, e restituire l’accuratezza media su (per esempio) 10 esecuzioni del programma

Opzionale: implementare corpus_lesk_algorithm utilizando semCor

In [31]:
import numpy as np
import random
import nltk
from nltk import MWETokenizer, WordNetLemmatizer
from nltk.corpus import wordnet as wn
#nltk.('sedownloadmcor')
from nltk.corpus import semcor
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

In [32]:
#show semcore
semcor.sents()[:100]


[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', 'Atlanta', "'s", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term', 'end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

In [33]:
import re
stop_words = set(stopwords.words('english')) #remove stop words
mwes = [x for x in wn.all_lemma_names() if '_' in x]
mwes = [tuple(x.split('_')) for x in mwes]
tokenizer = MWETokenizer(mwes, separator=' ')
lemmatizer = WordNetLemmatizer()

def preprocessing(text):
    text = re.sub(r'[^\w\s]',' ',text) #remove punctuation
    text = text.lower()
    text = tokenizer.tokenize(text.split())
    text = [lemmatizer.lemmatize(token) for token in text]
    text = [w for w in text if not w in stop_words]
    return text

In [34]:
semcor.tagged_sents(tag = 'sem')[:100]

[[['The'], Tree(Lemma('group.n.01.group'), [Tree('NE', ['Fulton', 'County', 'Grand', 'Jury'])]), Tree(Lemma('state.v.01.say'), ['said']), Tree(Lemma('friday.n.01.Friday'), ['Friday']), ['an'], Tree(Lemma('probe.n.01.investigation'), ['investigation']), ['of'], Tree(Lemma('atlanta.n.01.Atlanta'), ['Atlanta']), ["'s"], Tree(Lemma('late.s.03.recent'), ['recent']), Tree(Lemma('primary.n.01.primary_election'), ['primary', 'election']), Tree(Lemma('produce.v.04.produce'), ['produced']), ['``'], ['no'], Tree(Lemma('evidence.n.01.evidence'), ['evidence']), ["''"], ['that'], ['any'], Tree(Lemma('abnormality.n.04.irregularity'), ['irregularities']), Tree(Lemma('happen.v.01.take_place'), ['took', 'place']), ['.']], [['The'], Tree(Lemma('jury.n.01.jury'), ['jury']), Tree(Lemma('far.r.02.far'), ['further']), Tree(Lemma('state.v.01.say'), ['said']), ['in'], Tree(Lemma('term.n.02.term'), ['term']), Tree(Lemma('end.n.02.end'), ['end']), Tree(Lemma('presentment.n.01.presentment'), ['presentments']), ['

In [35]:
def lemma_list(sent):
    ret = []
    try:
        ret = [l.label() if isinstance(l, nltk.tree.Tree) else None for l in sent]
        ret = [get_synset_from_label(x) for x in ret if x is not None]
        ret = [x for x in ret if x is not None]
    except ValueError:
        print('error')

    return ret

def get_synset_from_label(label):
    if (len(str(label).split("'")) > 1):
        label = ''.join(str(label).split("'")[1:-1])
        try:
            return wn.synset('.'.join(str(label).split(".")[:-1]))
        except:
            return None
    else:
        return None

In [36]:
def get_context_from_semcor(target_sense):
    sentences = []
    
    #get the sentences in which the target sense is present
    for sent in semcor.tagged_sents(tag = 'sem'):
        if target_sense in lemma_list(sent):
            sentences.append(sent)

    return set(preprocessing(' '.join(sentences)))

In [37]:
def SimplifiedLesk(word, sentence):
    best_sense = wn.synsets(word)[0] if len(wn.synsets(word)) > 0 else None #wordnet ordina i sysnet in base alla frequenza
    max_overlap = 0
    context = set(sentence)
    
    for sense in wn.synsets(word):
        signature = set(preprocessing(sense.definition())).union(set(preprocessing(' '.join(sense.examples()))))
        overlap = len(context.intersection(signature))
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = sense
    
    return best_sense

def corpus_lesk(word, sentence):
    best_sense = wn.synsets(word)[0] if len(wn.synsets(word)) > 0 else None #wordnet ordina i sysnet in base alla frequenza
    max_overlap = 0
    context = set(sentence)
    
    for sense in wn.synsets(word):
        signature = set(preprocessing(sense.definition())).union(set(preprocessing(' '.join(sense.examples()))))
        signature = signature.union(get_context_from_semcor(sense))
        overlap = len(context.intersection(signature))
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = sense

    return best_sense

In [38]:
#get 50 random sentences from semcor
sentences_with_tag = semcor.tagged_sents(tag = 'sem')[:100]
sentences = semcor.sents()[:100]
np.random.seed(0)
indices = np.random.permutation(len(sentences_with_tag))
indices = indices[:50]

my_sentences_with_tag = [sentences_with_tag[i] for i in indices]
my_sentences = [sentences[i] for i in indices]

#print(my_sentences_with_tag[0])
#print(my_sentences[0])
num_correct = 0

for i, sentence in enumerate(my_sentences):
    #lowercase
    #sentence = [w.lower() for w in sentence]
    #remove stop words
    sentence_without_stopwords = [w for w in sentence if not w in stop_words]
    #remove punctuation
    sentence_without_stopwords = [w for w in sentence_without_stopwords if w.isalpha()]
    #get random word
    random_word = random.choice(sentence_without_stopwords)
    index = sentence.index(random_word)

    #print(random_word)

    actual_synset = []
    for j, cell in enumerate(my_sentences_with_tag[i]):
        if random_word in cell[0]:
            label = cell.label() if isinstance(cell, nltk.tree.Tree) else None
            if label:
                print('cell label: ', label)
                label = str(label).split("'")[1]
                actual_synset.append(wn.synset('.'.join(str(label).split(".")[:-1])))

    #run lesk
    predicted_synset = corpus_lesk(random_word, sentence)
    print('random word: ', random_word)
    print('predicted: ', predicted_synset)
    print('actual: ', actual_synset[0])
    print(predicted_synset,'==',actual_synset[0])
    print("correct: ", predicted_synset == actual_synset[0])
    num_correct += predicted_synset == actual_synset[0]
    print('-----------------')

print('accuracy: ',num_correct/len(my_sentences))

cell label:  Lemma('person.n.01.person')


TypeError: sequence item 0: expected str instance, SemcorSentence found